In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from utils.elasticsearch_utils import ElasticSearchUtils

/home/workspace/data-center/utils/elasticsearch_utils.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
def dump_docs(index):
    host_info = {
        'host': 'https://corpus.ncsoft.com:9200',
        'http_auth': 'elastic:nlplab',
    }

    utils = ElasticSearchUtils(**host_info)
    
    query = {
      '_source': [
        'title',
        'site',
        'property',
        'summary',
        '_list_doc.category',
        '_list_doc.name',
        '_list_doc.define',
        '_list_doc.detail_link',
      ]
    }

    doc_list = []
    utils.export(index=index, query=query, result=doc_list)    
    
    with open('data/{}.json.bz2'.format(index), 'w') as fp:
        for doc in tqdm(doc_list):
            line = json.dumps(doc, ensure_ascii=False)
            fp.write(line + '\n')    
    
    df = pd.DataFrame(doc_list)

    df.fillna('', inplace=True)
    
    return df

In [7]:
index = 'crawler-naver-terms-detail'

df = dump_docs(index=index)

print('{:,}'.format(len(df)))
df.head()

  1%|▏         | 18851/1363306 [00:00<00:14, 93554.22it/s]

100%|██████████| 1363306/1363306 [00:13<00:00, 98786.13it/s] 


1363306


,summary,site,property,_list_doc,title
0,요약 신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 8...,두산백과,출생-사망 ? ~ ?,{'detail_link': 'https://terms.naver.com/entry...,김면
1,요약 국가무형문화재 제47호인 궁시장(弓矢匠)의 기능보유자.,두산백과,출생-사망 1929.12.20 ~ 2011.4.11,{'detail_link': 'https://terms.naver.com/entry...,김박영
2,요약 일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 ...,두산백과,원어명 なつめ そうせき 출생-사망 1867.2.9 ~ 1916.12.9 본명 긴노스...,{'detail_link': 'https://terms.naver.com/entry...,나쓰메 소세키
3,요약 이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 ...,두산백과,원어명 نوال السعداوى 출생-사망 1931.10 27 ~ 국적 이집트 활동...,{'detail_link': 'https://terms.naver.com/entry...,나왈 엘 사다위
4,요약 한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스...,두산백과,출생-사망 1930. 09. 05. ~ 2017.12.20 본명 전봉선 활동분야 영...,{'detail_link': 'https://terms.naver.com/entry...,나애심


In [8]:
df['_list_doc'][0]

{'detail_link': 'https://terms.naver.com/entry.nhn?docId=1274324&cid=40942&categoryId=34295',
 'define': '신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 812년 시광록소경에 올라 조제책립부사에 충용되어 최릉을 따라 신라로 돌아왔다고 한다. 《해동역사》 인물고에 기록되어 있다. 신라의 문신으로 당나라에 볼모로 갔다. 812년(헌덕왕 4) 7월 시위위소경사자금어대(試衛尉少卿賜紫金魚袋)에서 시광록소경(試光祿少卿)에 올라 조제책립부사(吊祭冊立副使)에 충용(充用)되어 최릉(崔稜)을 따라 당나라에서 신라로 돌아왔다고 한다. 이 사실은 《해동역사 海東繹史》 인물고에 기록된 것으로 《삼국사기 三國史記》 헌덕왕 4년조에는 이같은 사실이 기록되어 있지 않다.',
 'name': '김면 [金沔, ? ~ ?]',
 'category': '두산백과/인물;두산백과/지역'}

In [9]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    for col in row['_list_doc']:
        df.at[i, col] = row['_list_doc'][col]
        
df.head()

100%|██████████| 1363306/1363306 [04:21<00:00, 5219.86it/s]


,summary,site,property,_list_doc,title,detail_link,define,name,category
0,요약 신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 8...,두산백과,출생-사망 ? ~ ?,{'detail_link': 'https://terms.naver.com/entry...,김면,https://terms.naver.com/entry.nhn?docId=127432...,신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 812년...,"김면 [金沔, ? ~ ?]",두산백과/인물;두산백과/지역
1,요약 국가무형문화재 제47호인 궁시장(弓矢匠)의 기능보유자.,두산백과,출생-사망 1929.12.20 ~ 2011.4.11,{'detail_link': 'https://terms.naver.com/entry...,김박영,https://terms.naver.com/entry.nhn?docId=129461...,국가무형문화재 제47호인 궁시장의 기능보유자. 1929년 경상북도 예천의 대대로 활...,"김박영 [金博榮, 1929.12.20 ~ 2011.4.11]",두산백과/인물
2,요약 일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 ...,두산백과,원어명 なつめ そうせき 출생-사망 1867.2.9 ~ 1916.12.9 본명 긴노스...,{'detail_link': 'https://terms.naver.com/entry...,나쓰메 소세키,https://terms.naver.com/entry.nhn?docId=107418...,일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 고답적...,"나쓰메 소세키 [Natsume Sōseki, 夏目漱石(하목수석), 1867.2.9 ...",두산백과/인물;두산백과/역사
3,요약 이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 ...,두산백과,원어명 نوال السعداوى 출생-사망 1931.10 27 ~ 국적 이집트 활동...,{'detail_link': 'https://terms.naver.com/entry...,나왈 엘 사다위,https://terms.naver.com/entry.nhn?docId=283592...,이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 책을 ...,"나왈 엘 사다위 [Nawal El Saadawi/Nawal al Saadawi, 1...",두산백과/인물
4,요약 한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스...,두산백과,출생-사망 1930. 09. 05. ~ 2017.12.20 본명 전봉선 활동분야 영...,{'detail_link': 'https://terms.naver.com/entry...,나애심,https://terms.naver.com/entry.nhn?docId=138110...,한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스키한 ...,"나애심 [羅愛心, 1930. 09. 05. ~ 2017.12.20]",두산백과/인물


In [10]:
df.drop(columns=['_list_doc'], inplace=True)

In [11]:
df.head()

,summary,site,property,title,detail_link,define,name,category
0,요약 신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 8...,두산백과,출생-사망 ? ~ ?,김면,https://terms.naver.com/entry.nhn?docId=127432...,신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 812년...,"김면 [金沔, ? ~ ?]",두산백과/인물;두산백과/지역
1,요약 국가무형문화재 제47호인 궁시장(弓矢匠)의 기능보유자.,두산백과,출생-사망 1929.12.20 ~ 2011.4.11,김박영,https://terms.naver.com/entry.nhn?docId=129461...,국가무형문화재 제47호인 궁시장의 기능보유자. 1929년 경상북도 예천의 대대로 활...,"김박영 [金博榮, 1929.12.20 ~ 2011.4.11]",두산백과/인물
2,요약 일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 ...,두산백과,원어명 なつめ そうせき 출생-사망 1867.2.9 ~ 1916.12.9 본명 긴노스...,나쓰메 소세키,https://terms.naver.com/entry.nhn?docId=107418...,일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 고답적...,"나쓰메 소세키 [Natsume Sōseki, 夏目漱石(하목수석), 1867.2.9 ...",두산백과/인물;두산백과/역사
3,요약 이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 ...,두산백과,원어명 نوال السعداوى 출생-사망 1931.10 27 ~ 국적 이집트 활동...,나왈 엘 사다위,https://terms.naver.com/entry.nhn?docId=283592...,이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 책을 ...,"나왈 엘 사다위 [Nawal El Saadawi/Nawal al Saadawi, 1...",두산백과/인물
4,요약 한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스...,두산백과,출생-사망 1930. 09. 05. ~ 2017.12.20 본명 전봉선 활동분야 영...,나애심,https://terms.naver.com/entry.nhn?docId=138110...,한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스키한 ...,"나애심 [羅愛心, 1930. 09. 05. ~ 2017.12.20]",두산백과/인물


In [26]:
df.fillna('', inplace=True)

In [30]:
def simple_category(x):
    result = []
    for v in x.split(';'):
        result.append(v.split('/', maxsplit=1)[-1])
    
    return ','.join(result)

df['category'] = df['category'].apply(lambda x: simple_category(x))

df.head()

,summary,site,property,title,detail_link,define,name,category
0,요약 신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 8...,두산백과,출생-사망 ? ~ ?,김면,https://terms.naver.com/entry.nhn?docId=127432...,신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 812년...,"김면 [金沔, ? ~ ?]","인물,지역"
1,요약 국가무형문화재 제47호인 궁시장(弓矢匠)의 기능보유자.,두산백과,출생-사망 1929.12.20 ~ 2011.4.11,김박영,https://terms.naver.com/entry.nhn?docId=129461...,국가무형문화재 제47호인 궁시장의 기능보유자. 1929년 경상북도 예천의 대대로 활...,"김박영 [金博榮, 1929.12.20 ~ 2011.4.11]",인물
2,요약 일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 ...,두산백과,원어명 なつめ そうせき 출생-사망 1867.2.9 ~ 1916.12.9 본명 긴노스...,나쓰메 소세키,https://terms.naver.com/entry.nhn?docId=107418...,일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 고답적...,"나쓰메 소세키 [Natsume Sōseki, 夏目漱石(하목수석), 1867.2.9 ...","인물,역사"
3,요약 이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 ...,두산백과,원어명 نوال السعداوى 출생-사망 1931.10 27 ~ 국적 이집트 활동...,나왈 엘 사다위,https://terms.naver.com/entry.nhn?docId=283592...,이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 책을 ...,"나왈 엘 사다위 [Nawal El Saadawi/Nawal al Saadawi, 1...",인물
4,요약 한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스...,두산백과,출생-사망 1930. 09. 05. ~ 2017.12.20 본명 전봉선 활동분야 영...,나애심,https://terms.naver.com/entry.nhn?docId=138110...,한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스키한 ...,"나애심 [羅愛心, 1930. 09. 05. ~ 2017.12.20]",인물


In [49]:
df[df['property'] == '[]']

,site,category,property,title,name,summary,define,detail_link
39,마블/DC 캐릭터백과,애니,,퍼니셔,퍼니셔 [Punisher],,[전쟁영웅] 뉴욕 브루클린에서 태어난 프랭크는 시칠리아 계통의 부모 밑에서 자랐다....,https://terms.naver.com/entry.nhn?docId=353683...
40,만화애니메이션사전,애니,,페그,"페그 [peg, ペグ]",,1. 레지스트레이션 페그(registration peg)의 준말. 2. 페그 바(p...,https://terms.naver.com/entry.nhn?docId=165130...
41,만화애니메이션사전,애니,,페이스,"페이스 [pace, ペ―ス, 速度]",,1. 내용 전개상의 속도나 리듬. 2. 필름이 영사되는 속도. 3. 장면 내의 연기...,https://terms.naver.com/entry.nhn?docId=165131...
42,만화애니메이션사전,애니,,장편 영화,"장편 영화 [feature film, フィーチャー フィルム, 長編]",,일반 영화관에서 상영할 목적으로 제작한 장편 영화. 엄밀히 정의되어 있지는 않지만 ...,https://terms.naver.com/entry.nhn?docId=165072...
43,만화애니메이션사전,애니,,페이드,"페이드 [fade, フェード]",,한 장면의 영상이 점차 어두워지면서 사라지거나 반대로 밝아지면서 선명해지는 광학 효...,https://terms.naver.com/entry.nhn?docId=165131...
...,...,...,...,...,...,...,...,...
1363300,군사용어사전,사회과학,,불발탄수색,"불발탄수색 [不發彈搜索, bomb reconnaissance]",,"불발탄을 수색하여 인원, 건물, 시설에 위해를 미치지 않도록 처치하고 그 성질을 확...",https://terms.naver.com/entry.nhn?docId=153617...
1363301,군사용어사전,사회과학,,불발핵폭탄,"불발핵폭탄 [不發核爆彈, nuclear dud]",,목표를 향하여 발사되거나 거치되었을 때 핵출력을 나타내도록 설계된 부분이 폭발을 일...,https://terms.naver.com/entry.nhn?docId=153617...
1363302,군사용어사전,사회과학,,복좌장치,"복좌장치 [複座裝置, counterrecoil mechanism]",,반동한 후 포를 적절한 사격자세로 돌려보내는 수압식 또는 기계적인 장치.,https://terms.naver.com/entry.nhn?docId=153604...
1363303,군사용어사전,사회과학,,복좌연습기,"복좌연습기 [複座練習機, training plane with dual controls]",,조종사와 조종사 훈련관이 사용하는 공동조종장치가 달린 연습기.,https://terms.naver.com/entry.nhn?docId=153603...


In [47]:
# df['property'] = df['property'].astype(str)
# df['summary'] = df['summary'].astype(str)

for i, row in tqdm(df.iterrows(), total=len(df)):
    for col in ['property', 'summary']:
        if row[col] == '[]':
            df.at[i, col] = ''

100%|██████████| 1363306/1363306 [02:21<00:00, 9617.67it/s] 


In [39]:
columns = ['site', 'category', 'property', 'title', 'name', 'summary', 'define', 'detail_link']

df = df[columns]
df.head()

,site,category,property,title,name,summary,define,detail_link
0,두산백과,"인물,지역",출생-사망 ? ~ ?,김면,"김면 [金沔, ? ~ ?]",요약 신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 8...,신라 후기의 문신이다. 당나라에 볼모로 가서 시위위소경사자금어대로 있던 중 812년...,https://terms.naver.com/entry.nhn?docId=127432...
1,두산백과,인물,출생-사망 1929.12.20 ~ 2011.4.11,김박영,"김박영 [金博榮, 1929.12.20 ~ 2011.4.11]",요약 국가무형문화재 제47호인 궁시장(弓矢匠)의 기능보유자.,국가무형문화재 제47호인 궁시장의 기능보유자. 1929년 경상북도 예천의 대대로 활...,https://terms.naver.com/entry.nhn?docId=129461...
2,두산백과,"인물,역사",원어명 なつめ そうせき 출생-사망 1867.2.9 ~ 1916.12.9 본명 긴노스...,나쓰메 소세키,"나쓰메 소세키 [Natsume Sōseki, 夏目漱石(하목수석), 1867.2.9 ...",요약 일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 ...,일본 소설가 겸 영문학자. 그의 작풍은 당시 전성기에 있던 자연주의에 대하여 고답적...,https://terms.naver.com/entry.nhn?docId=107418...
3,두산백과,인물,원어명 نوال السعداوى 출생-사망 1931.10 27 ~ 국적 이집트 활동...,나왈 엘 사다위,"나왈 엘 사다위 [Nawal El Saadawi/Nawal al Saadawi, 1...",요약 이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 ...,이집트의 여성운동가·작가·의사·심리학자이다. 이슬람 사회의 여성을 주제로 한 책을 ...,https://terms.naver.com/entry.nhn?docId=283592...
4,두산백과,인물,출생-사망 1930. 09. 05. ~ 2017.12.20 본명 전봉선 활동분야 영...,나애심,"나애심 [羅愛心, 1930. 09. 05. ~ 2017.12.20]",요약 한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스...,한국의 대중가수·영화배우. 1950~60년대 이국적인 외모와 탁음이 섞인 허스키한 ...,https://terms.naver.com/entry.nhn?docId=138110...


In [50]:
df.to_json(
    'data/{}.json.bz2'.format(index),
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

In [51]:
chunksize = 500000

writer = pd.ExcelWriter('data/{}.xlsx'.format(index))

for i, chunk in enumerate(np.split(df, len(df) // chunksize)):
    chunk.to_excel(writer, sheet_name='part-{}'.format(i), index=False)

writer.save()